In [112]:
import pandas as pd
import numpy as np
from datetime import datetime
import bql
from functools import reduce
from dateutil.relativedelta import relativedelta
import os
from pathlib import Path
bq=bql.Service()

In [142]:
enddate=datetime(2019,9,30)
startdate=datetime(1989,3,31)

In [143]:
def bqldf(req_item):
    req=bql.Request(memb, req_item)
    resp=bq.execute(req)
    df=resp[0].df().reset_index()
    return df

def bqldf_sec(sec, reqitem):
    req=bql.Request(sec, reqitem)
    resp=bq.execute(req)
    df=resp[0].df().reset_index()
    return df

In [144]:
def get_ratios(indx, std, edd):
    memb=bq.univ.members(indx)
    daterg_m=bq.func.range(std, edd, frq="M")
    daterg_a=bq.func.range(std.year,edd.year)
    params={
        'fa_period_reference' : bq.func.range(std.year, edd.year),
        'fa_period_type' : 'A',
        'fa_period_year_end' : 'F'
    }
    pbr={'PBR' : bq.data.px_to_book_ratio(dates=daterg_m, fa_period_year_end='F', fill='prev')}
    per={'PE_RATIO' : bq.data.pe_ratio(dates=daterg_m, fa_period_year_end='F', fill='prev')}
    px={'PX' : bq.data.px_last(dates=daterg_m, fill='prev')}
    dvd={'DVD' : bq.data.is_div_per_shr(**params)}
    
    per_df=bqldf(per).drop(['REVISION_DATE'], axis=1).rename(columns={'AS_OF_DATE' : 'DATE'})
    pbr_df=bqldf(pbr).drop(['REVISION_DATE', 'PERIOD_END_DATE'], axis=1).rename(columns={'AS_OF_DATE' : 'DATE'})
    px_df=bqldf(px).drop(['CURRENCY'], axis=1)
    daily_df=per_df.merge(pbr_df, on=['ID', 'DATE'])
    daily_df=daily_df.merge(px_df, on=['ID', 'DATE'])
    
    dvd_df=bqldf(dvd).drop(['CURRENCY', 'AS_OF_DATE', 'REVISION_DATE'], axis=1)
    df_all=daily_df.merge(dvd_df, on=['ID', 'PERIOD_END_DATE']).ffill()
    df_all['DVD_YIELD']=df_all['DVD']/df_all['PX']
    df_all=df_all[['ID', 'DATE', 'PERIOD_END_DATE', 'PE_RATIO', 'PBR', 'DVD_YIELD', 'DVD', 'PX']].dropna()
    
    return df_all

In [145]:
numyears=relativedelta(enddate, startdate).years
df=pd.DataFrame(columns=['ID', 'DATE', 'PERIOD_END_DATE', 'PE_RATIO', 'PBR', 'DVD_YIELD'])

for x in range(numyears):
    stdt=datetime(startdate.year + x, startdate.month, startdate.day)
    eddt=datetime(startdate.year + x+1, startdate.month, startdate.day)
    dftemp=get_ratios('TPX Index', stdt, eddt)
    df=df.append(dftemp).reset_index(drop=True)

filepath=os.getcwd()
filename='TPX' + '_' + str(startdate.strftime('%Y%m%d')) + '_' + str(enddate.strftime('%Y%m%d')) + ".csv"
pathname=str(Path(filepath + '/' + filename))
exporttocsv=df.to_csv(pathname)

BQL ERROR: Error: Daily limits exceeded, request is blocked., request_id=f77a4f679c98411892364e46b00d5f86 payload_id=None


ResponseError: Error: Daily limits exceeded, request is blocked.